In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [3]:
  df = fetch_datasets()['letter_img']
  X, y = df.data, df.target
  le = LabelEncoder()
  y = le.fit_transform(y)
  G = len(np.unique(y))
  print(np.shape(X), G, sum(y==0)*.6*.5, sum(y==1)*.6*.5)
  X[:3]

(20000, 16) 2 5779.8 220.2


array([[ 2.,  8.,  3.,  5.,  1.,  8., 13.,  0.,  6.,  6., 10.,  8.,  0.,
         8.,  0.,  8.],
       [ 5., 12.,  3.,  7.,  2., 10.,  5.,  5.,  4., 13.,  3.,  9.,  2.,
         8.,  4., 10.],
       [ 4., 11.,  6.,  8.,  6., 10.,  6.,  2.,  6., 10.,  3.,  7.,  3.,
         7.,  3.,  9.]])

In [3]:
missing_rates = np.array([0.5, 0.6, 0.7, 0.8, 0.9])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([-1000,800])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [4]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [5]:
leftindex = pd.MultiIndex.from_product([["50%", "60%", "70%", "80%", "90%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM         0.162$\pm$0.019  0.985$\pm$0.002   
                         LR          0.818$\pm$0.014  0.977$\pm$0.002   
                         DCT         0.436$\pm$0.061  0.971$\pm$0.004   
             sensitivity SVM         0.124$\pm$0.011  0.985$\pm$0.002   
                         LR           0.736$\pm$0.02  0.976$\pm$0.002   
                         DCT          0.317$\pm$0.05   0.97$\pm$0.005   
60%          F-1 score   SVM         0.072$\pm$0.018  0.981$\pm$0.002   
                         LR          0.777$\pm$0.009  0.973$\pm$0.003   
                         DCT         0.292$\pm$0.063  0.964$\pm$0.004   
             sensitivity SVM         0.072$\pm$0.009  0.982$\pm$0.002   
                         LR          0.678$\pm$0.013  0.972$\pm$0.003   
                         DCT         0.208$\pm$0.043  0.961$\pm$0.006   
70%          F-1 score   SVM         0.022$\pm$0.008  0.975$\pm$0.004   
                         LR          0.728$\pm$0.012  0.968$\pm$0.003   
                         DCT         0.114$\pm$0.065  0.945$\pm$0.007   
             sensitivity SVM         0.046$\pm$0.005  0.978$\pm$0.003   
                         LR          0.614$\pm$0.014  0.965$\pm$0.005   
                         DCT          0.097$\pm$0.04  0.938$\pm$0.009   
80%          F-1 score   SVM         0.012$\pm$0.004  0.967$\pm$0.005   
                         LR          0.683$\pm$0.013   0.95$\pm$0.011   
                         DCT         0.025$\pm$0.045  0.906$\pm$0.017   
             sensitivity SVM         0.043$\pm$0.002  0.974$\pm$0.003   
                         LR          0.561$\pm$0.015   0.94$\pm$0.016   
                         DCT          0.05$\pm$0.024  0.877$\pm$0.028   
90%          F-1 score   SVM         0.016$\pm$0.008  0.962$\pm$0.007   
                         LR           0.655$\pm$0.02  0.919$\pm$0.029   
                         DCT         0.018$\pm$0.029  0.847$\pm$0.024   
             sensitivity SVM         0.044$\pm$0.005   0.97$\pm$0.004   
                         LR          0.528$\pm$0.022  0.891$\pm$0.044   
                         DCT         0.045$\pm$0.016  0.783$\pm$0.038   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM         0.981$\pm$0.003  0.985$\pm$0.002   
                         LR          0.974$\pm$0.003  0.978$\pm$0.002   
                         DCT         0.974$\pm$0.003  0.972$\pm$0.003   
             sensitivity SVM          0.98$\pm$0.003  0.985$\pm$0.002   
                         LR          0.972$\pm$0.003  0.977$\pm$0.002   
                         DCT         0.974$\pm$0.004  0.971$\pm$0.003   
60%          F-1 score   SVM         0.981$\pm$0.003  0.983$\pm$0.002   
                         LR          0.971$\pm$0.003  0.974$\pm$0.003   
                         DCT         0.968$\pm$0.003  0.963$\pm$0.003   
             sensitivity SVM          0.98$\pm$0.003  0.984$\pm$0.002   
                         LR          0.969$\pm$0.004  0.973$\pm$0.003   
                         DCT         0.966$\pm$0.003  0.959$\pm$0.004   
70%          F-1 score   SVM         0.979$\pm$0.004  0.975$\pm$0.006   
                         LR          0.969$\pm$0.003  0.969$\pm$0.005   
                         DCT         0.956$\pm$0.007  0.947$\pm$0.008   
             sensitivity SVM          0.98$\pm$0.004  0.979$\pm$0.004   
                         LR          0.966$\pm$0.004  0.967$\pm$0.007   
                         DCT          0.952$\pm$0.01  0.939$\pm$0.011   
80%          F-1 score   SVM         0.968$\pm$0.006  0.968$\pm$0.006   
                         LR          0.955$\pm$0.011   0.95$\pm$0.013   
                         DCT         0.917$\pm$0.016  0.907$\pm$0.018   
             sensitivity SVM

In [6]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_letter_high.csv", index = False)

In [7]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
50\% & F-1 score & SVM &  0.162\$\textbackslash pm\$0.019 &  0.985\$\textbackslash pm\$0.002 &  0.981\$\textbackslash pm\$0.003 &  0.985\$\textbackslash pm\$0.002 &  0.942\$\textbackslash pm\$0.007 &  0.984\$\textbackslash pm\$0.002 \\
    &             & LR &  0.818\$\textbackslash pm\$0.014 &  0.977\$\textbackslash pm\$0.002 &  0.974\$\textbackslash pm\$0.003 &  0.978\$\textbackslash pm\$0.002 &  0.912\$\textbackslash pm\$0.012 &  0.978\$\textbackslash pm\$0.003 \\
    &             & DCT &  0.436\$\textbackslash pm\$0.061 &  0.971\$\textbackslash pm\$0.004 &  0.974\$\textbackslash pm\$0.003 &  0.972\$\textbackslash pm\$0.003 &  0.902\$\textbackslash p